In [126]:
# Run in Python 3.7 kernel
# Import libraries

import pandas as pd
import numpy as np # creating and manipulating arrays
import matplotlib.pyplot as plt # visualizing data
import sklearn # regression models
import statsmodels.api as sm 

1. Develop Linear Regression model using Statmodels

In [127]:
data = pd.read_csv("garments_worker_productivity.csv", header = 0) # read in data
data = pd.get_dummies(data, columns=["quarter","department","day","team"], drop_first=True) # create dummy variables for categorical data
data = data.dropna() # drop rows with NaN values
data = data.drop('date', axis=1) # drop dates
data_ols = sm.add_constant(data)
model = sm.OLS(endog= data_ols.actual_productivity, exog = data_ols.loc[:, data_ols.columns!='actual_productivity']) # use actual productivity as the dependent variable, and everything else as independent
results = model.fit()
print(results.summary())

# Compile Significant Variables
sigparams = results.params[results.pvalues < 0.05] # Get significant variables
sigparams = sigparams.to_frame() # Create a dataframe of them
sigparams.columns = ['coefficient'] # Name the columns
sigparams = sigparams['coefficient'].astype(float) # Convert to float so that you can sort it by absolute value
sigparams = sigparams.sort_values(key=abs, ascending=False) # Order by absolute value

                             OLS Regression Results                            
Dep. Variable:     actual_productivity   R-squared:                       0.813
Model:                             OLS   Adj. R-squared:                  0.805
Method:                  Least Squares   F-statistic:                     99.37
Date:                 Tue, 13 Sep 2022   Prob (F-statistic):          3.90e-219
Time:                         18:51:11   Log-Likelihood:                 889.27
No. Observations:                  691   AIC:                            -1719.
Df Residuals:                      661   BIC:                            -1582.
Df Model:                           29                                         
Covariance Type:             nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
targeted_productivit

2. Select the best features using forward selection

In [128]:
# Show significant parameters
print(sigparams)

targeted_productivity    0.621713
department_sweing        0.145683
team_11                 -0.043256
idle_men                -0.006253
smv                     -0.003984
incentive                0.003253
no_of_workers            0.002039
idle_time                0.000614
over_time               -0.000003
Name: coefficient, dtype: float64


Test Models

In [133]:
il = []
for x in range(0, (sigparams.count() -1)):
    if x == 0:
        model = sm.OLS(endog= data_ols.actual_productivity, exog = data_ols.loc[:, data_ols.columns==sigparams.index[0]])
        results = model.fit()
        print(results.summary())
        il.append(sigparams.index[0])
    else:
        il.append(sigparams.index[x])
        print(il)
        model = sm.OLS(endog= data_ols.actual_productivity, exog = data_ols.loc[:, data_ols.columns==[il]])
        results = model.fit()
        print(results.summary())




                                 OLS Regression Results                                 
Dep. Variable:     actual_productivity   R-squared (uncentered):                   0.977
Model:                             OLS   Adj. R-squared (uncentered):              0.977
Method:                  Least Squares   F-statistic:                          2.988e+04
Date:                 Tue, 13 Sep 2022   Prob (F-statistic):                        0.00
Time:                         18:58:00   Log-Likelihood:                          538.86
No. Observations:                  691   AIC:                                     -1076.
Df Residuals:                      690   BIC:                                     -1071.
Df Model:                            1                                                  
Covariance Type:             nonrobust                                                  
                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------

ValueError: ('Shapes must match', (32,), (1,))